In [10]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor
import torchvision.transforms
from tqdm import tqdm
import matplotlib.pyplot as plt
import torch.nn.functional as F

In [11]:
training_data = datasets.MNIST(
    root = "data",
    train = True,
    download = True,
    transform = ToTensor(),
)

test_data = datasets.MNIST(
    root = "data",
    train = False,
    download = True,
    transform = ToTensor(),
)

In [12]:
batch_size = 64

training_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)


for X, y in test_dataloader:
    print(f'Shape of X: [N, C, H, W]: {X.shape}')
    print(f'Shape of y: {y.shape}{y.type}')
    
    break

Shape of X: [N, C, H, W]: torch.Size([64, 1, 28, 28])
Shape of y: torch.Size([64])<built-in method type of Tensor object at 0x177912720>


In [13]:
device = 'cuda' if torch.cuda.is_available() else 'mps' if torch.backends.mps.is_available() else 'cpu'
print(f'Using: {device} device')

Using: mps device


In [18]:
class LeNet(nn.Module):
    def __init__(self) -> None:
        super().__init__()

        self.conv1 = nn.Conv2d(1, 6, 3)
        self.conv2 = nn.Conv2d(6, 16, 3)

        self.fc1 = nn.Linear(400, 512)
        self.fc2 = nn.Linear(512, 512)
        self.fc3 = nn.Linear(512, 512)
        self.fc4 = nn.Linear(512, 10)

    def forward(self, x):
        x = F.max_pool2d(F.relu(self.conv1(x)), (2,2))
        x = F.max_pool2d(F.relu(self.conv2(x)),2)
        x = x.view(-1, self.num_flat_features(x))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = self.fc4(x)
        return x

    def num_flat_features(self, x):
        size = x.size()[1:]
        num_features = 1

        for s in size:
            num_features *= s
        
        return num_features

model = LeNet().to(device=device)
print(model)

LeNet(
  (conv1): Conv2d(1, 6, kernel_size=(3, 3), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(3, 3), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=512, bias=True)
  (fc2): Linear(in_features=512, out_features=512, bias=True)
  (fc3): Linear(in_features=512, out_features=512, bias=True)
  (fc4): Linear(in_features=512, out_features=10, bias=True)
)


In [19]:
epochs = 10
learning_rate = 1e-3

loss_fn = nn.CrossEntropyLoss()
optim = torch.optim.Adam(model.parameters(), lr = learning_rate)

In [20]:
def train(dataloader, model, loss_fn, optimzer):
    model.train()

    for X,y in tqdm(dataloader):
        X, y = X.to(device), y.to(device)

        pred = model(X)
        loss = loss_fn(pred, y)

        optimzer.zero_grad()
        loss.backward()
        optimzer.step()

In [21]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    model.eval()

    correct = 0

    with torch.no_grad():

        for X, y in tqdm(dataloader):
            X, y = X.to(device), y.to(device)
            pred = model(X)

            x = (pred.argmax(1) == y)
            x = x.type(torch.float)
            x = x.sum()
            x = x.item()

            correct += x
    print(f'Accuracy(%): {correct/size*100}')

In [22]:
for i in range(epochs):
    train(training_dataloader, model, loss_fn, optim)
    test(test_dataloader, model, loss_fn)
    print(i)

100%|██████████| 157/157 [00:00<00:00, 245.18it/s]


Accuracy(%): 96.21
0


100%|██████████| 157/157 [00:00<00:00, 264.72it/s]


Accuracy(%): 98.21
1


100%|██████████| 157/157 [00:00<00:00, 261.78it/s]


...

Accuracy(%): 98.88
7


100%|██████████| 157/157 [00:00<00:00, 270.37it/s]


Accuracy(%): 98.74000000000001
8


100%|██████████| 157/157 [00:00<00:00, 271.32it/s]

Accuracy(%): 98.92
9


In [1]:
class ContaBancaria():

    contas = 0
    @classmethod
    def inc_inst(cls):
        cls.contas += 1
        print("Creating instance")

    def __init__(self, numero, nome, _saldo, tipo):
        ContaBancaria.inc_inst()
        self.numero = numero
        self.nome = nome
        self._saldo = _saldo
        self.tipo = tipo
        

    def deposito(self, qtd):
        self._saldo += qtd 
    
    def saque(self, qtd):
        self._saldo -= qtd
    
    def saldo(self):
        return self._saldo
    
    def __str__(self):
        return self.nome + " | saldo: " + str(self._saldo)
    
    @staticmethod
    def funcao_estatica():
        print("func_stat")
    
conta1 = ContaBancaria('001', 'Caetano', 10.05, 'corrente')
conta2 = ContaBancaria('002', 'Guilherme', 23.55, 'poupança')
print(conta2)
conta3 = ContaBancaria('003', 'Dalvan', 12.45, 'investimento')
print(conta1)
print(conta3)
conta1.deposito(44.45)
conta1.saque(33.33)
print('Saldo:', conta1.saldo())
print(f"instancias: {ContaBancaria.contas}")

Creating instance
Creating instance
Guilherme | saldo: 23.55
Creating instance
Caetano | saldo: 10.05
Dalvan | saldo: 12.45
Saldo: 21.17
instancias: 3


O objetivo é estender a classe `ContaBancaria` fornecendo três Subclasses (`ContaDeposito`, `ContaCorrente`, `ContaInvestimento`)
1. `ContaCorrente` precisa ter um limite de saque e redefinir o método para sacar do dinheiro. Este método precisa tratar se o saque for maior que o saldo.
2. `ContaDeposito` precisa ter uma taxa de juros associado 
3. `ContaInvestimento` precisa ter um atributo que identifica o tipo de investimento (ex. baixo risco e alto risco)
4. Testar o funcionamento

In [7]:
class ContaDeposito(ContaBancaria):
    def __init__(self, numero, nome, _saldo, juros, tipo="Deposito"):
        super().__init__(numero, nome, _saldo, tipo)
        self.juros=juros
    
    def depositar(self, deposito):
        self._saldo += deposito
        return self._saldo
    

class ContaCorrente(ContaBancaria):
    def __init__(self, numero, nome, _saldo, tipo="Corrente"):
        super().__init__(numero, nome, _saldo, tipo)

    def saque(self, saque):
        if saque > self._saldo:
            return "Not possible to withdraw"
        else:
            self._saldo -= saque
            return "Saque realizado"

    def __str__(self):
        return super().__str__() + " this is: Conta Corrente"
    
class ContaInvestimento(ContaBancaria):
    def __init__(self, numero, nome, _saldo, tipo="Investimento"):
        super().__init__(numero, nome, _saldo, tipo)

In [8]:
contajorge = ContaCorrente("001", "Jorge", 69)
print(contajorge)

Creating instance
Jorge | saldo: 69 this is: Conta Corrente


In [13]:
print(contajorge.saque(18))
print(contajorge.saldo())

Not possible to withdraw
15
